# Module 3: Creating a Simple CNN

## Practice: Training a Convolutional Neural Network on CIFAR-10

In this notebook, we will train a simple Convolutional Neural Network (CNN) on the CIFAR-10 dataset. CIFAR-10 consists of 60,000 32x32 color images across 10 classes (airplane, car, bird, cat, etc.).

Imagine an IoT device (e.g., Raspberry Pi + camera) capturing images and classifying them either on the device (edge computing) or in the cloud. This example demonstrates how a CNN learns to recognize objects in images.


## 1. Imports and Dataset Loading

We start by importing necessary libraries and loading the CIFAR-10 dataset directly from Keras. We'll also normalize pixel values to the [0,1] range.


Nesta celda importamos as librarías necesarias e cargamos o conxunto de datos CIFAR-10:

- **tensorflow**: Para construír e adestrar modelos de deep learning.
- **datasets, layers, models** de Keras: Para manexar datos e definir arquitecturas.
- **matplotlib.pyplot** e **numpy**: Para visualización e manipulación numérica.

Cargamos o CIFAR-10, que contén 60.000 imaxes de 32×32 en 10 clases, e normalizamos os píxeles dividindo entre 255. Finalmente, imprimimos as formas dos conxuntos de adestramento e proba.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

print("Training data shape:", x_train.shape)
print("Testing data shape:", x_test.shape)

## 2. Visualizing the Data

Let's look at a few samples from the training set to understand what the images look like. We'll display the first 5 images with their labels.


Nesta celda definimos os nomes das clases e mostramos as primeiras 5 imaxes de adestramento:

- Creamos unha lista `class_names` con etiquetas lexibles para cada clase.
- Usamos `plt.subplots` para crear unha fila de 5 subplots.
- Para cada imaxe, mostramos a imaxe, poñemos o título coa clase correspondente e ocultamos os eixes.

Isto serve para inspeccionar exemplos do conxunto de datos e entender a dispoñibilidade das clases.

In [ ]:
class_names = ["airplane","automobile","bird","cat","deer",
               "dog","frog","horse","ship","truck"]

# Show first 5 images
fig, axes = plt.subplots(1, 5, figsize=(10, 2))
for i in range(5):
    axes[i].imshow(x_train[i])
    label = y_train[i][0]  # because y_train[i] is an array
    axes[i].set_title(class_names[label])
    axes[i].axis('off')
plt.show()

## 3. Building the CNN Model

We will construct a simple CNN with two convolutional blocks followed by pooling layers, then flatten the output and add fully connected layers for classification.


Nesta celda definimos a arquitectura dunha Rede Convolucional (CNN) simple:

- Comezamos cun modelo **Sequential** de Keras.  
- Usamos unha capa **Input** explícita para indicar a forma dos datos de entrada (32×32×3).  
- Primeiro bloque convolucional: `Conv2D` con 32 filtros de tamaño 3×3 e activación **ReLU**, seguido de `MaxPooling2D` 2×2.  
- Segundo bloque convolucional: `Conv2D` con 64 filtros 3×3 e activación **ReLU**, seguido de `MaxPooling2D` 2×2.  
- Achatamos as características con `Flatten`.  
- Capas totalmente conectadas: unha `Dense` de 64 neuronas con **ReLU**, e a capa final `Dense` de 10 neuronas con **softmax** para clasificación en 10 clases.  
- Rematamos mostrando un resumo do modelo con `model.summary()`.  

In [ ]:
model = models.Sequential()
# Specify the input shape explicitly as the first layer
model.add(Input(shape=(32, 32, 3)))

# First convolutional block
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Second convolutional block
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 10 classes

model.summary()

## 4. Compiling and Training the Model

We'll compile the model using the Adam optimizer and sparse categorical crossentropy loss, then train for 5 epochs with a batch size of 64.


Nesta celda compilamos e adestramos o modelo definido anteriormente:

- **Optimizer:** Adam.
- **Loss Function:** sparse_categorical_crossentropy (axeitado para etiquetas enteiras).
- **Metrics:** accuracy.
- **Fit:** 5 épocas, tamaño de batch 64, usando test set para validación.

O obxecto `history` almacena a evolución da perda e precisión durante o adestramento e validación.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=5, 
                    validation_data=(x_test, y_test), 
                    batch_size=64)

## 5. Evaluating the Model

Evaluate the trained CNN on the test set to measure its accuracy.


Nesta celda avaliamos o modelo nos datos de proba:

- Chamamos `model.evaluate` con `x_test` e `y_test`.
- Obtemos `test_loss` e `test_acc`.
- Imprimimos a precisión final no conxunto de proba para cuantificar o rendemento.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

## 6. Making Predictions

Let's make predictions on a few test images and compare the predicted labels to the true labels.


Nesta celda realizamos predicións sobre as primeiras 5 imaxes de test:

- Usamos `model.predict` para obter as probabilidades de cada clase.
- Determinamos as etiquetas predicidas con `np.argmax`.
- Deseñamos unha figura con 5 imaxes e mostramos cada unha coa súa predición.

In [ ]:
# Prompt for the image URL
url = input("Image URL: ")

# Download and open the image
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# Resize to 32×32 and normalize pixel values to [0,1]
img = img.resize((32, 32))
img_array = np.array(img).astype('float32') / 255.0

# Create a batch of size 1
img_batch = np.expand_dims(img_array, axis=0)

# Make a prediction
preds = model.predict(img_batch)
class_idx = np.argmax(preds, axis=1)[0]
prob = preds[0][class_idx]

# Display the result
print(f"Predicted class: {class_names[class_idx]}  —  Probability: {prob:.2%}")

Nesta celda presentamos un exemplo de modelo estendido con regularización e plot da historia de adestramento:

- Definimos `model2` con bloques Conv2D, MaxPooling e un **Dropout** ao 25% para evitar sobreaxuste.
- Capa final Dense de 10 neuronas con softmax.
- Asumimos que almacenamos o obxecto `history2` tras adestrar o modelo extendido.
- Usamos `plt` para trazar dúas gráficas: precisión e perda de adestramento e validación ao longo das épocas.